In [ ]:
#| default_exp coord

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numba
import numpy as np
import math
from decorrelation.utils import ngjit,ngpjit
from numba import prange

In [ ]:
#| export
# test shows this automatical parallel can accelerate 3x, 
# probablely because the data calculting is accelerated but the data writing is not
@ngpjit
def _coords2idxs(coords, x0, dx, nx, out):
    '''input is 1d'''
    for i in prange(coords.shape[0]):
        _out = round((coords[i] - x0)/dx)
        if _out < 0: _out = 0
        if _out > nx-1: _out=nx-1
        out[i] = _out
    # np.round((coords - x0)/dx,out=out) # the test shows the automatical parallel is not fast enough
    # out[out < 0] = 0
    # out[out > nx - 1] = nx - 1

In [ ]:
#| export
@ngpjit
def _rasterize(pc, idx, ras, iidx):
    for i in prange(pc.shape[0]):
        yi = idx[0,i]; xi = idx[1,i]
        if iidx[yi,xi] == -1:
            ras[yi,xi,...] = pc[i,...] #in numba [y,x,:] is not supported
            iidx[yi,xi] = i

In [ ]:
#| export
class Coord(object):
    '''utils for digitize raster and point cloud data.
    The coord is defined as the continuous coordinates, e.g., longitude & latitude.
    The index is defined as the digitized index, (0,0,im,jm).
    '''
    def __init__(self,x0,dx,nx,y0,dy,ny):
        self.x0 = x0
        self.dx = dx
        self.nx = nx
        self.xm = x0+(nx-1)*dx
        self.y0 = y0
        self.dy = dy
        self.ny = ny
        self.ym = y0+(ny-1)*dy
        self.maxlevel = math.floor(math.log2(min(nx,ny)))
        self.p = math.ceil(math.log2(max(nx,ny))) # order of the hillbert curve
    
    def max_idx(self,level):
        return math.ceil(self.nx/(2**level))-1, math.ceil(self.ny/(2**level))-1
    
    def coord2idx(self,x,y,level): # include a buffer if not x, y not exactly on the grid
        xi = round((x-self.x0)/self.dx/2**level)
        yi = round((y-self.y0)/self.dy/2**level)
        xi_max, yi_max = self.max_idx(level)
        return sorted((0,xi,xi_max))[1], sorted((0,yi,yi_max))[1]
    
    def idx2coord(self,xi,yi,level):
        xi_max, yi_max = self.max_idx(level)
        xi, yi = sorted((0,xi,xi_max))[1], sorted((0,yi,yi_max))[1]
        return xi*2**level*self.dx+self.x0, yi*2**level*self.dy+self.y0
    
    def coords2idxs(self,coords):
        '''inputs are 2d arrays.'''
        out = np.empty_like(coords,dtype=np.int32)
        _coords2idxs(coords[1], self.x0, self.dx, self.nx, out[1])
        _coords2idxs(coords[0], self.y0, self.dy, self.ny, out[0])
        return out
    def rasterize(self,pc, # point cloud data, can be a stack, shape (n_pc,...)
                  idx, # 2D index of the pc, value within (0,0,im,jm), shape (2,n_pc)
                  fill_value=np.nan)->tuple: # the raster data, the index of the original point cloud list 
        ras = np.full((self.ny,self.nx,*pc.shape[1:]),fill_value=fill_value,dtype=pc.dtype)
        # print(ras)
        iidx = np.full((self.ny,self.nx),-1,dtype=np.int64)
        _rasterize(pc, idx, ras, iidx)
        return ras, iidx

In [ ]:
coord = Coord(1.8,0.2,10,-1.2,0.2,10)

pc = np.random.rand(5,3,2)
x = np.array([1.91,1.10,1.47,3.43,2.8])
y = np.array([-1.11,-1.09,-0.81,-0.4,0.11])
idx = coord.coords2idxs((y,x))
np.testing.assert_array_equal(idx, np.array([[0,1,2,4,7],[1,0,0,8,5]]))
# pc.shape, pc, idx

OMP: Info #277: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [ ]:
ras, iidx = coord.rasterize(pc,idx)
assert ras.shape[2:] == pc.shape[1:]
assert iidx.shape == (10,10)
for i in range(pc.shape[0]):
    _idx = np.argwhere(iidx==i)[0]
    np.testing.assert_array_equal(ras[_idx[0],_idx[1]], pc[i])
# ras[idx[1],idx[0]]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()